In [1]:
import numpy as np 
import pandas as pd 

In [39]:
def score(x_instance, class_weights):
    return np.dot(x_instance, class_weights)

#  true_labes is 0 and 1. 1 if class belongs
# loss = 0
# for x_instance in X:
#     all_scores = [score(x_instance, class_weights) for class_weights in all_weights]
#     exp_scores = [exp(class_score) for class_score in all_scores]
#     sum_of_exp_scores = sum(exp_scores)
#     probs = [exp_score / sum_of_exp_scores for exp_scores in exp_scores]

#     true labels and probs should be of the same order
#     cross_entropies = true_labels[c] * probs[c] for c in range(len(true_labels))
#     loss += sum(cross_entropies)
# loss = loss / X.shape

In [72]:
def predict_proba(X, weights):
    all_scores = [score(X, class_weights) for class_weights in weights]
    exp_scores = [np.exp(class_score) for class_score in all_scores]
    sum_of_exp_scores = sum(exp_scores)
    probs = [exp_score / sum_of_exp_scores for exp_score in exp_scores]

    return probs

In [41]:
pos_val = list(range(10))
neg_val = list(range(10, 20))
data = {}
y = []

for i in range(20):
    if i % 2 == 0:
        data[i] = pos_val
        y.append(0)
    else:
        data[i] = neg_val
        y.append(1)

X = pd.DataFrame(data).T.to_numpy()
y = np.array(y)

In [42]:
X

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [73]:
learning_rate = 0.1
classes = list(set(y))
weights = np.random.randn(len(classes), X.shape[1])

true_labels = [0 for c in classes]
gradients = np.array([np.zeros(X.shape[1]) for c in classes])

# как вариант можно отделать нужную часть X и работать со всем дф в numpy
# predict_proba(X, weights)
for x_instance, label in zip(X, y):
    probs = predict_proba(x_instance, weights)

    true_labels[label] = 1

    for c in classes:
        gradients[c] += (probs[c] - true_labels[c]) * x_instance 

gradients = gradients / X.shape[0]

# gradient shows direction in which the weights should be adjusted to minimize the loss.
weights = weights - learning_rate * gradients

[0 1 2 3 4 5 6 7 8 9] [[ 0.5088024  -0.64403217  1.80725518  0.81431167  0.35968123  0.29822828
  -1.81953666  1.07063348  1.02410053  0.40069055]
 [ 0.08402963 -0.81567655 -1.22580633 -1.25409739  0.30170287 -0.65865107
  -0.85042489 -0.16311124  1.01977673  1.3917282 ]]
[np.float64(16.719513079034975), np.float64(5.323414319644163)]
[10 11 12 13 14 15 16 17 18 19] [[ 0.5088024  -0.64403217  1.80725518  0.81431167  0.35968123  0.29822828
  -1.81953666  1.07063348  1.02410053  0.40069055]
 [ 0.08402963 -0.81567655 -1.22580633 -1.25409739  0.30170287 -0.65865107
  -0.85042489 -0.16311124  1.01977673  1.3917282 ]]
[np.float64(54.92085789799475), np.float64(-16.38188614056866)]
[0 1 2 3 4 5 6 7 8 9] [[ 0.5088024  -0.64403217  1.80725518  0.81431167  0.35968123  0.29822828
  -1.81953666  1.07063348  1.02410053  0.40069055]
 [ 0.08402963 -0.81567655 -1.22580633 -1.25409739  0.30170287 -0.65865107
  -0.85042489 -0.16311124  1.01977673  1.3917282 ]]
[np.float64(16.719513079034975), np.float64

In [ ]:
# weights -= learning_rate * gradient